## Generate [route].json

In [75]:
import json

In [77]:
data = json.load(open('../../../Desktop/taipei_taiwan_osm_line.geojson', encoding='utf-8', errors='ignore'))

In [78]:
def filter_line(features, name):
    return [item for item in features if item['properties']['name'] == name]

def near(x, y):
    dis = abs(x[0] - y[0]) + abs(x[1] - y[1])
    return dis < 0.001

def aggregate_coordinates(cords):
    for _ in range(len(cords)):
#         base = cords.pop()
        base = cords.pop(0)
        for cord in cords:
            if near(base[0], cord[-1]):
                base = cord[:-1] + base
                cords.pop(cords.index(cord))
            elif near(base[-1], cord[0]):
                base = base[:-1] + cord
                cords.pop(cords.index(cord))
#         cords.insert(0, base)
        cords.append(base)

    return cords

def aggregate_coordinates_reverse(cords):
    for _ in range(len(cords)):
        base = cords.pop(0)
        for cord in cords:
            if near(base[0], cord[0]):
                base = list(reversed(cord[1:])) + base
                cords.pop(cords.index(cord))
            elif near(base[-1], cord[-1]):
                base = base + list(reversed(cord[:-1]))
                cords.pop(cords.index(cord))
        cords.append(base)


    return cords

In [96]:
cords = [item['geometry']['coordinates'] for item in filter_line(data['features'], '捷運板南線')]

In [97]:
for _ in range(len(cords)):
    cords = aggregate_coordinates(cords)
for _ in range(len(cords)):
    cords = aggregate_coordinates_reverse(cords)

for r in cords:
    print(len(r), '---------------------\n', r)

423 ---------------------
 [[121.6185949, 25.0545705], [121.6175121, 25.0545335], [121.61719969999999, 25.0545362], [121.6167278, 25.0545001], [121.6162295, 25.0544542], [121.6153355, 25.0543723], [121.6142157, 25.0542527], [121.61387539999998, 25.0542114], [121.6133833, 25.0541404], [121.6125296, 25.053994], [121.6122531, 25.0539038], [121.6113877, 25.0535293], [121.61055760000002, 25.0530964], [121.6097851, 25.0527254], [121.6092111, 25.0524736], [121.60906350000002, 25.0524142], [121.6089116, 25.0523614], [121.60843019999999, 25.0522552], [121.6072821, 25.0521134], [121.60433589999998, 25.0517496], [121.6039396, 25.0517181], [121.59993239999999, 25.0512496], [121.5963255, 25.0510587], [121.5955454, 25.0509097], [121.5954615, 25.0508937], [121.59515170000002, 25.0508299], [121.59379829999999, 25.0505657], [121.59332589999998, 25.0504597], [121.5926561, 25.0503047], [121.5912687, 25.0500023], [121.5894727, 25.0495855], [121.587472, 25.0491275], [121.58670129999999, 25.0489532], [121.5

## Generate station.json

In [1]:
import csv
import json
import pandas

In [2]:
from itertools import islice

def window(seq, n=2):
    "Returns a sliding window (of width n) over data from the iterable"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result    
    for elem in it:
        result = result[1:] + (elem,)
        yield result

In [3]:
routes = {'BL':['blue_line.json'],
          'BR':['brown_line.json'],
          'R':['red_line.json'],
          'O':['orange_line1.json', 'orange_line2.json', 'orange_line3.json'],
          'G':['green_line.json']
        }
routes = {k: [json.load(open(r)) for r in v] for k, v in routes.items()}

In [4]:
routes

{'BL': [[[121.6185949, 25.0545705],
   [121.6175121, 25.0545335],
   [121.61719969999999, 25.0545362],
   [121.6167278, 25.0545001],
   [121.6162295, 25.0544542],
   [121.6153355, 25.0543723],
   [121.6142157, 25.0542527],
   [121.61387539999998, 25.0542114],
   [121.6133833, 25.0541404],
   [121.6125296, 25.053994],
   [121.6122531, 25.0539038],
   [121.6113877, 25.0535293],
   [121.61055760000002, 25.0530964],
   [121.6097851, 25.0527254],
   [121.6092111, 25.0524736],
   [121.60906350000002, 25.0524142],
   [121.6089116, 25.0523614],
   [121.60843019999999, 25.0522552],
   [121.6072821, 25.0521134],
   [121.60433589999998, 25.0517496],
   [121.6039396, 25.0517181],
   [121.59993239999999, 25.0512496],
   [121.5963255, 25.0510587],
   [121.5955454, 25.0509097],
   [121.5954615, 25.0508937],
   [121.59515170000002, 25.0508299],
   [121.59379829999999, 25.0505657],
   [121.59332589999998, 25.0504597],
   [121.5926561, 25.0503047],
   [121.5912687, 25.0500023],
   [121.5894727, 25.04958

In [5]:
station = pandas.DataFrame.from_csv('metro_station_raw.csv', index_col=None)

In [6]:
station

,station_code,construction_id,station_name_tw,station_name_en,line_code,line_name,zipcode,address,lat,lon
0,BR01,BR13,動物園,Taipei Zoo,BR,文湖線,11656,台北市文山區新光路二段32號,24.998197,121.579338
1,BR02,BR12,木柵,Muzha,BR,文湖線,11656,台北市文山區木柵路四段135號,24.998241,121.573145
2,BR03,BR11,萬芳社區,Wanfang Community,BR,文湖線,11653,台北市文山區萬芳路60號,24.998585,121.568102
3,BR04,BR10,萬芳醫院,Wanfang Hospital,BR,文湖線,11696,台北市文山區興隆路三段113號,24.999386,121.558152
4,BR05,BR9,辛亥,Xinhai,BR,文湖線,11694,台北市文山區辛亥路四段128號,25.005475,121.557107
5,BR06,BR8,麟光,Linguang,BR,文湖線,10676,台北市大安區和平東路三段410號,25.018535,121.558791
6,BR07,BR7,六張犁,Liuzhangli,BR,文湖線,10674,台北市大安區和平東路三段168號,25.023777,121.553115
7,BR08,BR6,科技大樓,Technology Building,BR,文湖線,10667,台北市大安區復興南路二段235號,25.026125,121.543437
8,BR09,BR5,大安,Daan,BR,文湖線,10683,台北市大安區信義路四段2號,25.032943,121.543551
9,BR10,BR4,忠孝復興,Zhongxiao Fuxing,BR,文湖線,10654,台北市大安區忠孝東路四段47號,25.041629,121.543767


In [7]:
from collections import namedtuple
station_col = ['line_code', 'station_name_tw', 'lat', 'lon']
Station = namedtuple('Station', station_col)

In [8]:
def near(x, y):
    dis = abs(x[0] - y[0]) + abs(x[1] - y[1])
    return dis < 0.003

In [14]:
%pdb
results = {}
for a, b in window(station[station_col].to_records(index=False)):
    a, b = Station(*a), Station(*b)
    if a.line_code == 'x' or b.line_code == 'x':
        continue
    if a.line_code != b.line_code:
        continue
    line_code = a.line_code
    aj, bj = -1, -1
    for lines in routes[line_code]:
        for j, point in enumerate(lines):
            if near([a.lon, a.lat], point):
                aj = j
            if near([b.lon, b.lat], point):
                bj = j

        if aj != -1 and bj != -1:
            segment = lines[aj:bj+1] if aj < bj else list(reversed(lines[bj:aj+1]))
            segment_rev = list(reversed(segment))
        
            results[f'{a.station_name_tw}{b.station_name_tw}'] = segment
            results[f'{b.station_name_tw}{a.station_name_tw}'] = segment_rev
            break
    if aj == -1 or bj == -1:
        print(f'[{a.station_name_tw} -> {b.station_name_tw}] segment not found.')

Automatic pdb calling has been turned ON


In [15]:
results

{'七張大坪林': [[121.5427192, 24.9778881],
  [121.5425399, 24.9784335],
  [121.54233519999998, 24.9790299],
  [121.5421803, 24.9795023],
  [121.54203089999999, 24.9800919],
  [121.54187610000001, 24.9805805],
  [121.5417674, 24.9810477],
  [121.5415673, 24.9816453],
  [121.541281, 24.9829389],
  [121.5412153, 24.9834258],
  [121.54106459999998, 24.9840766]],
 '七張新店區公所': [[121.5427192, 24.9778881],
  [121.5428097, 24.9774429],
  [121.5428695, 24.976714],
  [121.5429278, 24.9759504],
  [121.5429395, 24.9755131],
  [121.542946, 24.9750607],
  [121.5429416, 24.9745107],
  [121.54293069999999, 24.9737378],
  [121.542925, 24.9732811],
  [121.54292959999998, 24.9729632],
  [121.54291069999998, 24.9719695],
  [121.54277919999998, 24.9714692],
  [121.5426495, 24.9710496],
  [121.542582, 24.9707918],
  [121.5423407, 24.9701133],
  [121.54137119999999, 24.9673828]],
 '三和國中三重國小': [[121.4863886, 25.0768253],
  [121.48809169999998, 25.0758493],
  [121.4907078, 25.0744347],
  [121.4920347, 25.0736544],
  

In [16]:
with open('stations.json', 'w') as f:
    json.dump(results, f)